In [1]:
!pip install torchcsprng==0.1.3+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install opacus

In [3]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
import os
import opacus
from opacus import PrivacyEngine

In [4]:
# __all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
#            'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
#            'wide_resnet50_2', 'wide_resnet101_2']


# model_urls = {
#     'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
#     'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
#     'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
#     'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
#     'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
#     'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
#     'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
#     'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
#     'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
# }

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, k, kernel_size = 5, stride = 1):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels*k, kernel_size, stride)
        self.tanh = nn.Tanh()
        self.pool = nn.AvgPool2d(kernel_size = 2)
    
    def forward(self, x):
        x = self.conv(x)
        out = self.pool(self.tanh(x))
        
        return out 

### Vanilla LeNet

In [6]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

In [7]:
from torchsummary import summary
model = LeNet5(10)
model.cuda()
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             Tanh-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/ba

### Wide LeNet with Linear Bottle Neck

In [8]:
class Linear_WideLeNet5(nn.Module):

    def __init__(self, n_classes):
        super(Linear_WideLeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=32, kernel_size=5, stride=1, padding = 1),
            nn.Conv2d(in_channels=32, out_channels=8, kernel_size=1, stride=1, padding = 1),
            nn.Conv2d(in_channels=8, out_channels=62, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=62, out_channels=11, kernel_size=1, stride=1, padding = 0),
            nn.Conv2d(in_channels=11, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        # print(x.shape)
        x = torch.flatten(x, 1)
        # print(x.shape)
        logits = self.classifier(x)
        return logits

In [9]:
from torchsummary import summary
Linear_WideLeNet5 = Linear_WideLeNet5(10)
Linear_WideLeNet5.cuda()
summary(Linear_WideLeNet5, (3, 32, 32))

torch.Size([2, 120, 1, 1])
torch.Size([2, 120])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           4,832
            Conv2d-5            [-1, 8, 14, 14]             264
            Conv2d-6           [-1, 62, 10, 10]          12,462
              Tanh-7           [-1, 62, 10, 10]               0
         AvgPool2d-8             [-1, 62, 5, 5]               0
            Conv2d-9             [-1, 11, 5, 5]             693
           Conv2d-10            [-1, 120, 1, 1]          33,120
             Tanh-11            [-1, 120, 1, 1]               0
           Linear-12                   [-1, 84]          10,164
             Tanh-13                   [-1, 84]        

### Wide LeNet with Non-Linear Bottle Neck

In [10]:
class Non_Linear_WideLeNet5(nn.Module):

    def __init__(self, n_classes):
        super(Non_Linear_WideLeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=32, kernel_size=5, stride=1, padding = 1),
            nn.Tanh(),
            nn.Conv2d(in_channels=32, out_channels=8, kernel_size=1, stride=1, padding = 1),
            nn.Tanh(),
            nn.Conv2d(in_channels=8, out_channels=62, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=62, out_channels=11, kernel_size=1, stride=1, padding = 0),
            nn.Tanh(),
            nn.Conv2d(in_channels=11, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        # print(x.shape)
        x = torch.flatten(x, 1)
        # print(x.shape)
        logits = self.classifier(x)
        return logits

In [11]:
from torchsummary import summary
Non_Linear_WideLeNet5 = Non_Linear_WideLeNet5(10)
Non_Linear_WideLeNet5.cuda()
summary(Non_Linear_WideLeNet5, (3, 32, 32))

torch.Size([2, 120, 1, 1])
torch.Size([2, 120])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           4,832
              Tanh-5           [-1, 32, 12, 12]               0
            Conv2d-6            [-1, 8, 14, 14]             264
              Tanh-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 62, 10, 10]          12,462
              Tanh-9           [-1, 62, 10, 10]               0
        AvgPool2d-10             [-1, 62, 5, 5]               0
           Conv2d-11             [-1, 11, 5, 5]             693
             Tanh-12             [-1, 11, 5, 5]               0
           Conv2d-13            [-1, 120, 1, 1]        

In [12]:
best_acc = []
best_train_acc = best_test_acc = start_epoch = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
optimizer1 = optim.Adam(Linear_WideLeNet5.parameters(), lr = 0.001)

In [13]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [14]:
def train(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [15]:
def test(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(f'Epoch {epoch} Step {batch_idx}/{len(testloader)}', 'Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_test_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_test_acc = acc

In [16]:
def train_wide(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer1.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer1.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [19]:
batch_size = sample_size = 32

privacy_engine = PrivacyEngine(
    model,
    batch_size,
    sample_size,
    alphas=[10, 100],
    noise_multiplier= 1.3,
    max_grad_norm=1.0,
)
privacy_engine.attach(optimizer)

for epoch in range(start_epoch, start_epoch+50):
    train(model, epoch)
    test(model, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Epoch 0 Step 0/1563 Loss: 2.301 | Acc: 12.500% (4/32)
Epoch 0 Step 1/1563 Loss: 2.301 | Acc: 12.500% (8/64)
Epoch 0 Step 2/1563 Loss: 2.301 | Acc: 11.458% (11/96)
Epoch 0 Step 3/1563 Loss: 2.304 | Acc: 8.594% (11/128)
Epoch 0 Step 4/1563 Loss: 2.303 | Acc: 10.625% (17/160)
Epoch 0 Step 5/1563 Loss: 2.304 | Acc: 9.375% (18/192)
Epoch 0 Step 6/1563 Loss: 2.305 | Acc: 8.929% (20/224)
Epoch 0 Step 7/1563 Loss: 2.305 | Acc: 8.594% (22/256)
Epoch 0 Step 8/1563 Loss: 2.305 | Acc: 8.681% (25/288)
Epoch 0 Step 9/1563 Loss: 2.306 | Acc: 8.438% (27/320)
Epoch 0 Step 10/1563 Loss: 2.305 | Acc: 9.091% (32/352)
Epoch 0 Step 11/1563 Loss: 2.304 | Acc: 10.417% (40/384)
Epoch 0 Step 12/1563 Loss: 2.303 | Acc: 10.337% (43/416)
Epoch 0 Step 13/1563 Loss: 2.303 | Acc: 11.161% (50/448)
Epoch 0 Step 14/1563 Loss: 2.303 | Acc: 11.250% (54/480)
Epoch 0 Step 15/1563 Loss: 2.303 | Acc: 10.938% (56/512)
Epoch 0 Step 16/1563 Loss: 2.302 | Acc: 11.397% (62/544)
Epoch 0 Step 17/1563 Loss: 2.302 | Acc: 11.458% (66/5

/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:283: UserWarning: PrivacyEngine expected a batch of size 32 but the last step received a batch of size 16. This means that the privacy analysis will be a bit more pessimistic. You can set `drop_last = True` in your PyTorch dataloader to avoid this problem completely
  f"PrivacyEngine expected a batch of size {self.batch_size} "


Streaming output truncated to the last 5000 lines.
Epoch 47 Step 632/1563 Loss: 2.083 | Acc: 32.469% (6577/20256)
Epoch 47 Step 633/1563 Loss: 2.083 | Acc: 32.472% (6588/20288)
Epoch 47 Step 634/1563 Loss: 2.082 | Acc: 32.500% (6604/20320)
Epoch 47 Step 635/1563 Loss: 2.083 | Acc: 32.488% (6612/20352)
Epoch 47 Step 636/1563 Loss: 2.083 | Acc: 32.501% (6625/20384)
Epoch 47 Step 637/1563 Loss: 2.082 | Acc: 32.509% (6637/20416)
Epoch 47 Step 638/1563 Loss: 2.082 | Acc: 32.526% (6651/20448)
Epoch 47 Step 639/1563 Loss: 2.082 | Acc: 32.510% (6658/20480)
Epoch 47 Step 640/1563 Loss: 2.082 | Acc: 32.518% (6670/20512)
Epoch 47 Step 641/1563 Loss: 2.082 | Acc: 32.511% (6679/20544)
Epoch 47 Step 642/1563 Loss: 2.083 | Acc: 32.528% (6693/20576)
Epoch 47 Step 643/1563 Loss: 2.083 | Acc: 32.512% (6700/20608)
Epoch 47 Step 644/1563 Loss: 2.083 | Acc: 32.524% (6713/20640)
Epoch 47 Step 645/1563 Loss: 2.084 | Acc: 32.508% (6720/20672)
Epoch 47 Step 646/1563 Loss: 2.084 | Acc: 32.501% (6729/20704)
Epoc

In [21]:
# Training Wide_ResNet16
best_train_acc = best_test_acc = 0
privacy_engine = PrivacyEngine(
    Linear_WideLeNet5,
    batch_size,
    sample_size,
    alphas=[10, 100],
    noise_multiplier=1.3,
    max_grad_norm=1.0,
)
privacy_engine.attach(optimizer1)

for epoch in range(start_epoch, start_epoch+50):
  train_wide(Linear_WideLeNet5, epoch)
  test(Linear_WideLeNet5, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 0/1563 Loss: 2.276 | Acc: 12.500% (4/32)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 1/1563 Loss: 2.285 | Acc: 9.375% (6/64)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 2/1563 Loss: 2.292 | Acc: 8.333% (8/96)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 3/1563 Loss: 2.296 | Acc: 9.375% (12/128)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 4/1563 Loss: 2.298 | Acc: 8.125% (13/160)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 5/1563 Loss: 2.299 | Acc: 7.812% (15/192)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 6/1563 Loss: 2.297 | Acc: 8.929% (20/224)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 7/1563 Loss: 2.296 | Acc: 9.766% (25/256)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 Step 8/1563 Loss: 2.297 | Acc: 9.375% (27/288)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 0 St

/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:283: UserWarning: PrivacyEngine expected a batch of size 32 but the last step received a batch of size 16. This means that the privacy analysis will be a bit more pessimistic. You can set `drop_last = True` in your PyTorch dataloader to avoid this problem completely
  f"PrivacyEngine expected a batch of size {self.batch_size} "


Streaming output truncated to the last 5000 lines.
torch.Size([32, 120])
Epoch 49 Step 209/1563 Loss: 2.100 | Acc: 28.051% (1885/6720)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 210/1563 Loss: 2.101 | Acc: 28.051% (1894/6752)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 211/1563 Loss: 2.100 | Acc: 28.066% (1904/6784)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 212/1563 Loss: 2.101 | Acc: 28.052% (1912/6816)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 213/1563 Loss: 2.101 | Acc: 28.052% (1921/6848)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 214/1563 Loss: 2.101 | Acc: 28.067% (1931/6880)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 215/1563 Loss: 2.100 | Acc: 28.154% (1946/6912)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 216/1563 Loss: 2.098 | Acc: 28.211% (1959/6944)
torch.Size([32, 120, 1, 1])
torch.Size([32, 120])
Epoch 49 Step 217/1563 Loss: 2.

In [22]:
print(best_acc)

[[53.125, 37.36], [50.0, 35.77]]
